In [2]:
from datasets import load_dataset

dataset = load_dataset("imagefolder", data_files="./Custom_data.zip")

In [6]:
labels = dataset["train"].features["label"].names
label2id, id2label = dict(), dict()
for i, label in enumerate(labels):
    label2id[label] = i
    id2label[i] = label

In [8]:
splits = dataset["train"].train_test_split(test_size=0.1)
train_dataset = splits['train']
test_dataset = splits['test']

In [12]:
from transformers import AutoImageProcessor, AutoModelForImageClassification

model_checkpoint = "Neruoy/swin-finetuned-food101-e3"
batch_size = 32

image_processor  = AutoImageProcessor.from_pretrained(model_checkpoint)
image_processor 

ViTImageProcessor {
  "do_normalize": true,
  "do_rescale": true,
  "do_resize": true,
  "feature_extractor_type": "ViTFeatureExtractor",
  "image_mean": [
    0.485,
    0.456,
    0.406
  ],
  "image_processor_type": "ViTImageProcessor",
  "image_std": [
    0.229,
    0.224,
    0.225
  ],
  "resample": 3,
  "rescale_factor": 0.00392156862745098,
  "size": {
    "height": 224,
    "width": 224
  }
}

In [13]:
from torchvision.transforms import (
    Normalize,
    Resize,
    CenterCrop,
    Compose,
    ToTensor,
)

normalize = Normalize(mean=image_processor.image_mean, std=image_processor.image_std)
if "height" in image_processor.size:
    size = (image_processor.size["height"], image_processor.size["width"])
    crop_size = size
    max_size = None
    print('Height')
elif "shortest_edge" in image_processor.size:
    size = image_processor.size["shortest_edge"]
    crop_size = (size, size)
    max_size = image_processor.size.get("longest_edge")
    print('Shortest')

train_transforms = Compose(
        [
            Resize(size),
            CenterCrop(crop_size),
            ToTensor(),
            normalize,
        ]
    )

test_transfroms = Compose(
        [
            Resize(size),
            CenterCrop(crop_size),
            ToTensor(),
            normalize,
        ]
    )

def preprocess_train(example_batch):
    example_batch["pixel_values"] = [
        train_transforms(image.convert("RGB")) for image in example_batch["image"]
    ]
    return example_batch

def preprocess_test(example_batch):
    example_batch["pixel_values"] = [test_transfroms(image.convert("RGB")) for image in example_batch["image"]]
    return example_batch

Height


In [14]:
train_dataset.set_transform(preprocess_train)
test_dataset.set_transform(preprocess_test)

In [16]:
model = AutoModelForImageClassification.from_pretrained(
    model_checkpoint, 
    label2id=label2id,
    id2label=id2label,
    ignore_mismatched_sizes = True, 
)

Some weights of SwinForImageClassification were not initialized from the model checkpoint at Neruoy/swin-finetuned-food101-e3 and are newly initialized because the shapes did not match:
- classifier.weight: found shape torch.Size([101, 1024]) in the checkpoint and torch.Size([10, 1024]) in the model instantiated
- classifier.bias: found shape torch.Size([101]) in the checkpoint and torch.Size([10]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [17]:
from torchinfo import summary

summary(model=model, 
        input_size=(32, 3, 224, 224),
        col_names=["input_size", "output_size", "num_params", "trainable"],
        col_width=20,
        row_settings=["var_names"])

Layer (type (var_name))                                                          Input Shape          Output Shape         Param #              Trainable
SwinForImageClassification (SwinForImageClassification)                          [32, 3, 224, 224]    [32, 10]             --                   True
├─SwinModel (swin)                                                               [32, 3, 224, 224]    [32, 1024]           --                   True
│    └─SwinEmbeddings (embeddings)                                               [32, 3, 224, 224]    [32, 3136, 128]      --                   True
│    │    └─SwinPatchEmbeddings (patch_embeddings)                               [32, 3, 224, 224]    [32, 3136, 128]      6,272                True
│    │    └─LayerNorm (norm)                                                     [32, 3136, 128]      [32, 3136, 128]      256                  True
│    │    └─Dropout (dropout)                                                    [32, 3136, 128]     

In [18]:
for param in model.base_model.parameters():
    param.requires_grad = False

In [19]:
from torchinfo import summary

summary(model=model, 
        input_size=(32, 3, 224, 224),
        col_names=["input_size", "output_size", "num_params", "trainable"],
        col_width=20,
        row_settings=["var_names"])

Layer (type (var_name))                                                          Input Shape          Output Shape         Param #              Trainable
SwinForImageClassification (SwinForImageClassification)                          [32, 3, 224, 224]    [32, 10]             --                   Partial
├─SwinModel (swin)                                                               [32, 3, 224, 224]    [32, 1024]           --                   False
│    └─SwinEmbeddings (embeddings)                                               [32, 3, 224, 224]    [32, 3136, 128]      --                   False
│    │    └─SwinPatchEmbeddings (patch_embeddings)                               [32, 3, 224, 224]    [32, 3136, 128]      (6,272)              False
│    │    └─LayerNorm (norm)                                                     [32, 3136, 128]      [32, 3136, 128]      (256)                False
│    │    └─Dropout (dropout)                                                    [32, 3136, 12

In [20]:
from transformers import TrainingArguments, Trainer

args = TrainingArguments(
    model_checkpoint,
    remove_unused_columns=False,
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    learning_rate=1e-3,
    per_device_train_batch_size=batch_size,
    gradient_accumulation_steps=4,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=3,
    warmup_ratio=0.1,
    logging_steps=10,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    push_to_hub=False,
)

In [21]:
import evaluate
import numpy as np

metric = evaluate.load('accuracy')

def compute_metrics(eval_pred):
    predictions = np.argmax(eval_pred.predictions, axis=1)
    return metric.compute(predictions=predictions, references=eval_pred.label_ids)

In [22]:
import torch

def collate_fn(examples):
    pixel_values = torch.stack([example["pixel_values"] for example in examples])
    labels = torch.tensor([example["label"] for example in examples])
    return {"pixel_values": pixel_values, "labels": labels}

In [23]:
trainer = Trainer(
    model,
    args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=image_processor,
    compute_metrics=compute_metrics,
    data_collator=collate_fn,
)

In [27]:
model.to('mps:0')
model.device

device(type='mps', index=0)

In [28]:
train_results = trainer.train()

  0%|          | 0/210 [00:00<?, ?it/s]

{'loss': 1.8353, 'learning_rate': 0.0004761904761904762, 'epoch': 0.14}
{'loss': 0.2833, 'learning_rate': 0.0009523809523809524, 'epoch': 0.28}
{'loss': 0.041, 'learning_rate': 0.0009523809523809524, 'epoch': 0.43}
{'loss': 0.0636, 'learning_rate': 0.0008994708994708994, 'epoch': 0.57}
{'loss': 0.0457, 'learning_rate': 0.0008465608465608465, 'epoch': 0.71}
{'loss': 0.0525, 'learning_rate': 0.0007936507936507937, 'epoch': 0.85}
{'loss': 0.043, 'learning_rate': 0.0007407407407407407, 'epoch': 0.99}


  0%|          | 0/32 [00:00<?, ?it/s]

{'eval_loss': 0.03788638487458229, 'eval_accuracy': 0.989, 'eval_runtime': 35.9026, 'eval_samples_per_second': 27.853, 'eval_steps_per_second': 0.891, 'epoch': 0.99}
{'loss': 0.0251, 'learning_rate': 0.0006878306878306878, 'epoch': 1.13}
{'loss': 0.031, 'learning_rate': 0.0006349206349206349, 'epoch': 1.28}
{'loss': 0.0339, 'learning_rate': 0.000582010582010582, 'epoch': 1.42}
{'loss': 0.0409, 'learning_rate': 0.0005291005291005291, 'epoch': 1.56}
{'loss': 0.0225, 'learning_rate': 0.0004761904761904762, 'epoch': 1.7}
{'loss': 0.0494, 'learning_rate': 0.00042328042328042324, 'epoch': 1.84}
{'loss': 0.0316, 'learning_rate': 0.00037037037037037035, 'epoch': 1.99}


  0%|          | 0/32 [00:00<?, ?it/s]

{'eval_loss': 0.04139889404177666, 'eval_accuracy': 0.99, 'eval_runtime': 36.577, 'eval_samples_per_second': 27.34, 'eval_steps_per_second': 0.875, 'epoch': 2.0}
{'loss': 0.0173, 'learning_rate': 0.00031746031746031746, 'epoch': 2.13}
{'loss': 0.0273, 'learning_rate': 0.00026455026455026457, 'epoch': 2.27}
{'loss': 0.0256, 'learning_rate': 0.00021164021164021162, 'epoch': 2.41}
{'loss': 0.0172, 'learning_rate': 0.00015873015873015873, 'epoch': 2.55}
{'loss': 0.0281, 'learning_rate': 0.00010582010582010581, 'epoch': 2.7}
{'loss': 0.0273, 'learning_rate': 5.2910052910052905e-05, 'epoch': 2.84}
{'loss': 0.0172, 'learning_rate': 0.0, 'epoch': 2.98}


  0%|          | 0/32 [00:00<?, ?it/s]

{'eval_loss': 0.035387810319662094, 'eval_accuracy': 0.991, 'eval_runtime': 35.9349, 'eval_samples_per_second': 27.828, 'eval_steps_per_second': 0.891, 'epoch': 2.98}
{'train_runtime': 1101.7038, 'train_samples_per_second': 24.507, 'train_steps_per_second': 0.191, 'train_loss': 0.1313641648207392, 'epoch': 2.98}


In [29]:
trainer.save_model('./Mymodel/')
trainer.log_metrics("train", train_results.metrics)
trainer.save_metrics("train", train_results.metrics)
trainer.save_state()

***** train metrics *****
  epoch                    =       2.98
  train_loss               =     0.1314
  train_runtime            = 0:18:21.70
  train_samples_per_second =     24.507
  train_steps_per_second   =      0.191


In [32]:
metrics = trainer.evaluate()
trainer.log_metrics("eval", metrics)
trainer.save_metrics("eval", metrics)

  0%|          | 0/32 [00:00<?, ?it/s]

***** eval metrics *****
  epoch                   =       2.98
  eval_accuracy           =      0.991
  eval_loss               =     0.0354
  eval_runtime            = 0:00:36.99
  eval_samples_per_second =     27.033
  eval_steps_per_second   =      0.865
